In [5]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input


class EmotionDetector:
    def __init__(self, model_path, cascade_path):
        # Load the emotion detection model
        self.model = load_model(model_path)
        # Mapping of emotion labels to their corresponding indices
        self.emotions_mapping = {0: 'anger',
                                 1: 'contempt',
                                 2: 'disgust',
                                 3: 'fear',
                                 4: 'happy',
                                 5: 'neutral',
                                 6: 'sad',
                                 7: 'surprise',
                                 8: 'uncertain'}
        # Load the face cascade classifier
        self.face_cascade = cv2.CascadeClassifier(cascade_path)

    def preprocess_frame(self, face_roi):
        # Resize the face region of interest (ROI) to the required input size
        face_resized = cv2.resize(face_roi, (224, 224))
        # Convert the resized face ROI to RGB and normalize pixel values
        face_resized_rgb = cv2.cvtColor(face_resized, cv2.COLOR_BGR2RGB)
        img_array = img_to_array(face_resized_rgb)
        img_array = preprocess_input(img_array)
        face = np.expand_dims(img_array, axis=0)
        return face

    def predict_emotion(self, face_roi):
        # Preprocess the face ROI
        preprocessed_frame = self.preprocess_frame(face_roi)
        # Make predictions using the emotion detection model
        predictions = self.model(preprocessed_frame)
        # Get the predicted emotion label
        predicted_emotion = self.emotions_mapping[np.argmax(predictions)]
        return predicted_emotion

    def process_video(self, input_path, output_path):
        # Open the video file for reading
        video_capture = cv2.VideoCapture(input_path)
        # Get the frames per second of the video
        fps = video_capture.get(cv2.CAP_PROP_FPS)
        # Open the video file for reading
        frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Create a VideoWriter object to save the processed video
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

        # Process each frame in the video
        while True:
            # Read a frame from the video
            ret, frame = video_capture.read()
            if not ret:
                break

            # Convert the frame to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # Detect faces in the grayscale frame
            faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)

            # Process each detected face
            for (x, y, w, h) in faces:
                # Extract the face region of interest (ROI)
                face_roi = frame[y:y+h, x:x+w]
                # Extract the face region of interest (ROI)
                emotion = self.predict_emotion(face_roi)
                # Draw a bounding box around the face and display the predicted emotion
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            out.write(frame)

        video_capture.release()
        out.release()
        cv2.destroyAllWindows()

In [2]:
# Define the paths for the emotion detection model and face cascade classifier
model_path = 'best_model.h5' # Path to CV-model
cascade_path = 'haarcascade_frontalface_default.xml' # Path to "Haar Cascades"-xml file

# Initialize the emotion detector with the model and cascade paths
emotion_detector = EmotionDetector(model_path, cascade_path)

In [4]:
# Process the video and save the processed video with bounding boxes and emotions
input_video_path = 'input_video.mp4'
output_video_path = 'output_video.mp4'
emotion_detector.process_video(input_video_path, output_video_path)